<a name="top"></a><img src="./source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)   

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## Assignement overlap
SpinalHDL will check that no signal assignement completly erases a previous one.   
The following code will throw the following error:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  a := 66 //Erease the a := 42 :(
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  val something = in Bool()
  when(something){
    a := 66
  }
}
showRtl(new TopLevel)

But in the case you really want to override the previous assignement (Yes, it could make sense in some cases), you can do the following:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  a.allowOverride
  a := 66
}
showRtl(new TopLevel)

## Clock crossing violation
SpinalHDL will check that every register of your design only depends (through some combinatorial logic) on registers which use the same or a synchronous clock domain. 

The following code will throw error:

In [ ]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")

  val regA = clkA(Reg(UInt(8 bits)))   //PlayDev.scala:834
  val regB = clkB(Reg(UInt(8 bits)))   //PlayDev.scala:835

  val tmp = regA + regA                //PlayDev.scala:838
  regB := tmp
}
showRtl(new TopLevel)

There are multiple possible fixes:
### crossClockDomain tag
The crossClockDomain tag can be used to say “It’s alright, don’t panic” to SpinalHDL

In [ ]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")

  val regA = clkA(Reg(UInt(8 bits)))
  val regB = clkB(Reg(UInt(8 bits))).addTag(crossClockDomain)


  val tmp = regA + regA
  regB := tmp
}
showRtl(new TopLevel)

### setSyncronousWith
You can also specify that two clock domains are syncronous together.

In [ ]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")
  clkB.setSyncronousWith(clkA)

  val regA = clkA(Reg(UInt(8 bits)))
  val regB = clkB(Reg(UInt(8 bits)))


  val tmp = regA + regA
  regB := tmp
}

showRtl(new TopLevel)

### BufferCC
Signal Bits or Gray-coded Bits can use BufferCC to cross different clockDomain

In [ ]:
class syncRead2Write extends Component {
  val io = new Bundle{
    val pushClock, pushRst = in Bool()
    val readPtr = in UInt(8 bits)
  }
  val pushCC = new ClockingArea(ClockDomain(io.pushClock, io.pushRst)) {
    val pushPtrGray = RegNext(toGray(io.readPtr)) init(0)
  }
}
showRtl(new syncRead2Write)

## Combinational loop
SpinalHDL will check that there are no combinatorial loops in the design.

The following code will throw error:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits) //PlayDev.scala line 831
  val b = UInt(8 bits) //PlayDev.scala line 832
  val c = UInt(8 bits)
  val d = UInt(8 bits)

  a := b
  b := c | d
  d := a
  c := 0
}
showRtl(new TopLevel)

A possible fix could be:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits) //PlayDev.scala line 831
  val b = UInt(8 bits) //PlayDev.scala line 832
  val c = UInt(8 bits)
  val d = UInt(8 bits)

  a := b
  b := c | d
  d := 42
  c := 0
}
showRtl(new TopLevel)

### False-positive
It should be said that SpinalHDL’s algorithm to detect combinatorial loops can be pessimistic, and it may give false positives. If it is giving a false positive, you can manualy disable loop checking on one signal of the loop like so:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 0
  a(1) := a(0) //False positive because of this line
}
showRtl(new TopLevel)

could be fixed by :

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits).noCombLoopCheck
  a := 0
  a(1) := a(0)
}
showRtl(new TopLevel)

It should also be said that assignments such as (a(1) := a(0)) can make some tools like Verilator unhappy. It may be better to use a Vec(Bool(), 8) in this case

## Hierarchy violation

SpinalHDL will check that signals are never accessed outside of the current component’s scope.

The following signals can be read inside a component:

- All directionless signals defined in the current component

- All in/out/inout signals of the current component

- All in/out/inout signals of children components

In addition, the following signals can be assigned to inside a component:

- All directionless signals defined in the current component

- All out/inout signals of the current component

- All in/inout signals of children components

If a `HIERARCHY VIOLATION`error appears, it means that one of the above rules was violated.

The following code will throw error:

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in UInt(8 bits)
  }
  val tmp = U"x42"
  io.a := tmp
}
showRtl(new TopLevel)

A fix could be :

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = out UInt(8 bits) // changed from in to out
  }
  val tmp = U"x42"
  io.a := tmp
}
showRtl(new TopLevel)

## Io bundle
SpinalHDL will check that each io bundle contains only in/out/inout signals.

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = UInt(8 bits)
  }
}
showRtl(new TopLevel)